In [1]:
from mailbox import _PartialFile
import numpy as np
import xarray as xr
import os
import re
from cdo import *
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind
import skill_metrics as sm
from brokenaxes import brokenaxes

reload(ca)

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo()

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof
import statsmodels.api as sm
import metpy.constants as constants

# 对historical数据去除长期趋势

## 读取historical的数据

In [2]:
hgthis_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/zg_historical_r144x72_195001-201412.nc")
uhis_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/ua_historical_r144x72_195001-201412.nc")
vhis_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/va_historical_r144x72_195001-201412.nc")
qhis_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/hus_historical_r144x72_195001-201412.nc")
waphis_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/wap_historical_r144x72_195001-201412.nc")
tahis_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/ta_historical_r144x72_195001-201412.nc")
prehis_ds_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/pr_historical_r144x72_195001-201412.nc")
sphis_ds_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/ps_historical_r144x72_195001-201412.nc")
tshis_ds_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/ts_historical_r144x72_195001-201412.nc")
toshis_ds_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/tos_historical_r144x72_195001-201412.nc")
hflshis_ds_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/hfls_historical_r144x72_195001-201412.nc")

In [3]:
toshis_ds_MAM = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/non_detrend/tos_historical_r144x72_197901-201412.nc")
toshis_ds_SON = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/non_detrend/tos_historical_r144x72_197901-201412.nc")
toshis_ds_DJF = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/non_detrend/tos_historical_r144x72_197901-201412.nc")

waphis_ds_MAM = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/non_detrend/wap_historical_r144x72_197901-201412.nc")
waphis_ds_SON = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/non_detrend/wap_historical_r144x72_197901-201412.nc")
waphis_ds_DJF = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/non_detrend/wap_historical_r144x72_197901-201412.nc")

## 重新计算去趋势结果

In [4]:
# 选取1979-2014年的结果
hgthis_ds_ver_JJA_detrend = ca.detrend_dim(hgthis_ds_ver_JJA.sel(time=hgthis_ds_ver_JJA.time.dt.year>=1979))
uhis_ds_ver_JJA_detrend = ca.detrend_dim(uhis_ds_ver_JJA.sel(time=uhis_ds_ver_JJA.time.dt.year>=1979))
vhis_ds_ver_JJA_detrend = ca.detrend_dim(vhis_ds_ver_JJA.sel(time=vhis_ds_ver_JJA.time.dt.year>=1979))
qhis_ds_ver_JJA_detrend = ca.detrend_dim(qhis_ds_ver_JJA.sel(time=qhis_ds_ver_JJA.time.dt.year>=1979))
waphis_ds_ver_JJA_detrend = ca.detrend_dim(waphis_ds_ver_JJA.sel(time=waphis_ds_ver_JJA.time.dt.year>=1979))
tahis_ds_ver_JJA_detrend = ca.detrend_dim(tahis_ds_ver_JJA.sel(time=tahis_ds_ver_JJA.time.dt.year>=1979))
prehis_ds_JJA_detrend = ca.detrend_dim(prehis_ds_JJA.sel(time=prehis_ds_JJA.time.dt.year>=1979))
sphis_ds_JJA_detrend = ca.detrend_dim(sphis_ds_JJA.sel(time=sphis_ds_JJA.time.dt.year>=1979))
tshis_ds_JJA_detrend = ca.detrend_dim(tshis_ds_JJA.sel(time=tshis_ds_JJA.time.dt.year>=1979))
toshis_ds_JJA_detrend = ca.detrend_dim(toshis_ds_JJA.sel(time=toshis_ds_JJA.time.dt.year>=1979))
hflshis_ds_JJA_detrend = ca.detrend_dim(hflshis_ds_JJA.sel(time=hflshis_ds_JJA.time.dt.year>=1979))

/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/home/soft/anaconda3/envs/chenhj_test_ca

In [5]:
toshis_ds_MAM_detrend = ca.detrend_dim(toshis_ds_MAM.sel(time=toshis_ds_MAM.time.dt.year>=1979))
toshis_ds_SON_detrend = ca.detrend_dim(toshis_ds_SON.sel(time=toshis_ds_SON.time.dt.year>=1979))
toshis_ds_DJF_detrend = ca.detrend_dim(toshis_ds_DJF.sel(time=toshis_ds_DJF.time.dt.year>=1979))

waphis_ds_MAM_detrend = ca.detrend_dim(waphis_ds_MAM.sel(time=waphis_ds_MAM.time.dt.year>=1979))
waphis_ds_SON_detrend = ca.detrend_dim(waphis_ds_SON.sel(time=waphis_ds_SON.time.dt.year>=1979))
waphis_ds_DJF_detrend = ca.detrend_dim(waphis_ds_DJF.sel(time=waphis_ds_DJF.time.dt.year>=1979))

/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/home/soft/anaconda3/envs/chenhj_test_cartopy/lib/python3.8/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])


In [6]:
hgthis_ds_ver_JJA_detrend.name = "zg"
uhis_ds_ver_JJA_detrend.name = "ua"
vhis_ds_ver_JJA_detrend.name = "va"
qhis_ds_ver_JJA_detrend.name = "hus"
waphis_ds_ver_JJA_detrend.name = "wap"
tahis_ds_ver_JJA_detrend.name = "ta"
prehis_ds_JJA_detrend.name = "pr"
sphis_ds_JJA_detrend.name = "sp"
tshis_ds_JJA_detrend.name = "ts"
toshis_ds_JJA_detrend.name = "tos"
hflshis_ds_JJA_detrend.name = "hfls"

In [7]:
toshis_ds_MAM_detrend.name = "tos"
toshis_ds_SON_detrend.name = "tos"
toshis_ds_DJF_detrend.name = "tos"
waphis_ds_MAM_detrend.name = "wap"
waphis_ds_SON_detrend.name = "wap"
waphis_ds_DJF_detrend.name = "wap"

In [8]:
hgthis_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/hgt_historical_r144x72_detrend.nc")
uhis_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/u_historical_r144x72_detrend.nc")
vhis_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/v_historical_r144x72_detrend.nc")
qhis_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/q_historical_r144x72_detrend.nc")
waphis_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/wap_historical_r144x72_detrend.nc")
tahis_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/ta_historical_r144x72_detrend.nc")
prehis_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/pre_historical_r144x72_detrend.nc")
sphis_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/sp_historical_r144x72_detrend.nc")
tshis_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/ts_historical_r144x72_detrend.nc")
toshis_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/tos_historical_r144x72_detrend.nc")
hflshis_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/hfls_historical_r144x72_detrend.nc")

In [9]:
toshis_ds_MAM.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/tos_historical_r144x72_detrend.nc")
toshis_ds_SON.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/tos_historical_r144x72_detrend.nc")
toshis_ds_DJF.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/tos_historical_r144x72_detrend.nc")
waphis_ds_MAM.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/wap_historical_r144x72_detrend.nc")
waphis_ds_SON.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/wap_historical_r144x72_detrend.nc")
waphis_ds_DJF.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/wap_historical_r144x72_detrend.nc")

# 对ssp245数据去除长期趋势

## 读取ssp245的数据

In [ ]:
hgthis_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/zg_historical_r144x72_195001-201412.nc")

In [ ]:
hgt_ssp245_path = (
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/zg"
)
g = os.walk(hgt_ssp245_path)
filepath = []
modelname_hgt = []
for path, dir_list, file_name in g:
    for filename in file_name:
        if re.search("ensemble", filename) == None:
            filepath.append(os.path.join(path, filename))
            loc = ca.retrieve_allstrindex(filename, "_")
            modelname_hgt.append(filename[loc[1] + 1 : loc[2]])
hgtds_ssp245 = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
hgtssp245_ds = xr.DataArray(hgtds_ssp245["zg"])
hgtssp245_ds.coords["models"] = modelname_hgt

u_ssp245_path = (
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/ua"
)
g = os.walk(u_ssp245_path)
filepath = []
modelname_u = []
for path, dir_list, file_name in g:
    for filename in file_name:
        if re.search("ensemble", filename) == None:
            filepath.append(os.path.join(path, filename))
            loc = ca.retrieve_allstrindex(filename, "_")
            modelname_u.append(filename[loc[1] + 1 : loc[2]])
uds_ssp245 = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
ussp245_ds = xr.DataArray(uds_ssp245["ua"])
ussp245_ds.coords["models"] = modelname_u

v_ssp245_path = (
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/va"
)
g = os.walk(v_ssp245_path)
filepath = []
modelname_v = []
for path, dir_list, file_name in g:
    for filename in file_name:
        if re.search("ensemble", filename) == None:
            filepath.append(os.path.join(path, filename))
            loc = ca.retrieve_allstrindex(filename, "_")
            modelname_v.append(filename[loc[1] + 1 : loc[2]])
vds_ssp245 = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
vssp245_ds = xr.DataArray(vds_ssp245["va"])
vssp245_ds.coords["models"] = modelname_v

sp_ssp245_path = (
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/ps"
)
g = os.walk(sp_ssp245_path)
filepath = []
modelname_sp = []
for path, dir_list, file_name in g:
    for filename in file_name:
        if re.search("ensemble", filename) == None:
            filepath.append(os.path.join(path, filename))
            loc = ca.retrieve_allstrindex(filename, "_")
            modelname_sp.append(filename[loc[1] + 1 : loc[2]])
spds_ssp245 = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
spssp245_ds = xr.DataArray(spds_ssp245["ps"])
spssp245_ds.coords["models"] = modelname_sp

q_ssp245_path = (
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/hus"
)
g = os.walk(q_ssp245_path)
filepath = []
modelname_q = []
for path, dir_list, file_name in g:
    for filename in file_name:
        if re.search("ensemble", filename) == None:
            filepath.append(os.path.join(path, filename))
            loc = ca.retrieve_allstrindex(filename, "_")
            modelname_q.append(filename[loc[1] + 1 : loc[2]])
qds_ssp245 = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
qssp245_ds = xr.DataArray(qds_ssp245["hus"])
qssp245_ds.coords["models"] = modelname_q

pr_ssp245_path = (
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/pr"
)
g = os.walk(pr_ssp245_path)
filepath = []
modelname_pr = []
for path, dir_list, file_name in g:
    for filename in file_name:
        if re.search("ensemble", filename) == None:
            filepath.append(os.path.join(path, filename))
            loc = ca.retrieve_allstrindex(filename, "_")
            modelname_pr.append(filename[loc[1] + 1 : loc[2]])
preds_ssp245 = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
pressp245_ds = xr.DataArray(preds_ssp245["pr"])*3600*24
pressp245_ds.coords["models"] = modelname_pr

wap_ssp245_path = (
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/wap"
)
g = os.walk(wap_ssp245_path)
filepath = []
modelname_wap = []
for path, dir_list, file_name in g:
    for filename in file_name:
        if re.search("ensemble", filename) == None:
            filepath.append(os.path.join(path, filename))
            loc = ca.retrieve_allstrindex(filename, "_")
            modelname_wap.append(filename[loc[1] + 1 : loc[2]])
wapds_ssp245 = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
wapssp245_ds = xr.DataArray(wapds_ssp245["wap"])
wapssp245_ds.coords["models"] = modelname_wap

ta_ssp245_path = (
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/ta"
)
g = os.walk(ta_ssp245_path)
filepath = []
modelname_ta = []
for path, dir_list, file_name in g:
    for filename in file_name:
        if re.search("ensemble", filename) == None:
            filepath.append(os.path.join(path, filename))
            loc = ca.retrieve_allstrindex(filename, "_")
            modelname_ta.append(filename[loc[1] + 1 : loc[2]])
tads_ssp245 = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
tassp245_ds = xr.DataArray(tads_ssp245["ta"])
tassp245_ds.coords["models"] = modelname_ta

ts_ssp245_path = (
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/ts"
)
g = os.walk(ts_ssp245_path)
filepath = []
modelname_ts = []
for path, dir_list, file_name in g:
    for filename in file_name:
        if re.search("ensemble", filename) == None:
            filepath.append(os.path.join(path, filename))
            loc = ca.retrieve_allstrindex(filename, "_")
            modelname_ts.append(filename[loc[1] + 1 : loc[2]])
tsds_ssp245 = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
tsssp245_ds = xr.DataArray(tsds_ssp245["ts"])
tsssp245_ds.coords["models"] = modelname_ts
tos_ssp245_path = (
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tos"
)
g = os.walk(tos_ssp245_path)
filepath = []
modelname_tos = []
for path, dir_list, file_name in g:
    for filename in file_name:
        if re.search("ensemble", filename) == None:
            filepath.append(os.path.join(path, filename))
            loc = ca.retrieve_allstrindex(filename, "_")
            modelname_tos.append(filename[loc[1] + 1 : loc[2]])
tosds_ssp245 = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
tosssp245_ds = xr.DataArray(tosds_ssp245["tos"])
tosssp245_ds.coords["models"] = modelname_tos
hfls_ssp245_path = (
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/hfls"
)
g = os.walk(hfls_ssp245_path)
filepath = []
modelname_hfls = []
for path, dir_list, file_name in g:
    for filename in file_name:
        if re.search("ensemble", filename) == None:
            filepath.append(os.path.join(path, filename))
            loc = ca.retrieve_allstrindex(filename, "_")
            modelname_hfls.append(filename[loc[1] + 1 : loc[2]])
hflsds_ssp245 = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
hflsssp245_ds = xr.DataArray(hflsds_ssp245["hfls"])
hflsssp245_ds.coords["models"] = modelname_hfls

## 处理垂直坐标系 

In [ ]:
hgtssp245_ds.coords["plev"] = hgtssp245_ds["plev"] / 100.0
hgtssp245_ds = hgtssp245_ds.rename({"plev": "level"})
ussp245_ds.coords["plev"] = ussp245_ds["plev"] / 100.0
ussp245_ds = ussp245_ds.rename({"plev": "level"})
vssp245_ds.coords["plev"] = vssp245_ds["plev"] / 100.0
vssp245_ds = vssp245_ds.rename({"plev": "level"})
qssp245_ds.coords["plev"] = qssp245_ds["plev"] / 100.0
qssp245_ds = qssp245_ds.rename({"plev": "level"})
wapssp245_ds.coords["plev"] = wapssp245_ds["plev"] / 100.0
wapssp245_ds = wapssp245_ds.rename({"plev": "level"})
tassp245_ds.coords["plev"] = tassp245_ds["plev"] / 100.0
tassp245_ds = tassp245_ds.rename({"plev": "level"})

## 选择季节

In [ ]:
hgtssp245_ds_ver_JJA = ca.p_time(hgtssp245_ds, 6, 8, True)
ussp245_ds_ver_JJA = ca.p_time(ussp245_ds, 6, 8, True)
vssp245_ds_ver_JJA = ca.p_time(vssp245_ds, 6, 8, True)
qssp245_ds_ver_JJA = ca.p_time(qssp245_ds, 6, 8, True)

wapssp245_ds_ver_MAM, wapssp245_ds_ver_JJA, wapssp245_ds_ver_SON, wapssp245_ds_ver_DJF = ca.mon_to_season(wapssp245_ds)

tassp245_ds_ver_JJA = ca.p_time(tassp245_ds, 6, 8, True)
spssp245_ds_JJA = ca.p_time(spssp245_ds, 6, 8, True)
pressp245_ds_JJA = ca.p_time(pressp245_ds, 6, 8, True)
tsssp245_ds_JJA = ca.p_time(tsssp245_ds, 6, 8, True)

tosssp245_ds_MAM, tosssp245_ds_JJA, tosssp245_ds_SON, tosssp245_ds_DJF = ca.mon_to_season(tosssp245_ds)

hflsssp245_ds_JJA = ca.p_time(hflsssp245_ds, 6, 8, True)

## 对各个变量模式的顺序进行重排

In [ ]:
models=hgthis_ds_ver_JJA.coords["models"]

hgtssp245_ds_ver_JJA_copy = hgtssp245_ds_ver_JJA.copy()
ussp245_ds_ver_JJA_copy = ussp245_ds_ver_JJA.copy()
vssp245_ds_ver_JJA_copy = vssp245_ds_ver_JJA.copy()
qssp245_ds_ver_JJA_copy = qssp245_ds_ver_JJA.copy()
wapssp245_ds_ver_MAM_copy = wapssp245_ds_ver_MAM.copy()
wapssp245_ds_ver_JJA_copy = wapssp245_ds_ver_JJA.copy()
wapssp245_ds_ver_SON_copy = wapssp245_ds_ver_SON.copy()
wapssp245_ds_ver_DJF_copy = wapssp245_ds_ver_DJF.copy()
tassp245_ds_ver_JJA_copy = tassp245_ds_ver_JJA.copy()
spssp245_ds_JJA_copy = spssp245_ds_JJA.copy()
pressp245_ds_JJA_copy = pressp245_ds_JJA.copy()
tsssp245_ds_JJA_copy = tsssp245_ds_JJA.copy()
tosssp245_ds_MAM_copy = tosssp245_ds_MAM.copy()
tosssp245_ds_JJA_copy = tosssp245_ds_JJA.copy()
tosssp245_ds_SON_copy = tosssp245_ds_SON.copy()
tosssp245_ds_DJF_copy = tosssp245_ds_DJF.copy()
hflsssp245_ds_JJA_copy = hflsssp245_ds_JJA.copy()


for i, mod in enumerate(models):
    hgtssp245_ds_ver_JJA_copy[i, :, :, :, :] = np.array(hgtssp245_ds_ver_JJA.sel(models=mod))
    ussp245_ds_ver_JJA_copy[i, :, :, :, :] = np.array(ussp245_ds_ver_JJA.sel(models=mod))
    vssp245_ds_ver_JJA_copy[i, :, :, :, :] = np.array(vssp245_ds_ver_JJA.sel(models=mod))
    qssp245_ds_ver_JJA_copy[i, :, :, :, :] = np.array(qssp245_ds_ver_JJA.sel(models=mod))
    
    wapssp245_ds_ver_MAM_copy[i, :, :, :, :] = np.array(wapssp245_ds_ver_MAM.sel(models=mod))
    wapssp245_ds_ver_JJA_copy[i, :, :, :, :] = np.array(wapssp245_ds_ver_JJA.sel(models=mod))
    wapssp245_ds_ver_SON_copy[i, :, :, :, :] = np.array(wapssp245_ds_ver_SON.sel(models=mod))
    wapssp245_ds_ver_DJF_copy[i, :, :, :, :] = np.array(wapssp245_ds_ver_DJF.sel(models=mod))
    
    tassp245_ds_ver_JJA_copy[i, :, :, :, :] = np.array(tassp245_ds_ver_JJA.sel(models=mod))
    spssp245_ds_JJA_copy[i, :, :, :] = np.array(spssp245_ds_JJA.sel(models=mod))
    pressp245_ds_JJA_copy[i, :, :, :] = np.array(pressp245_ds_JJA.sel(models=mod))
    tsssp245_ds_JJA_copy[i, :, :, :] = np.array(tsssp245_ds_JJA.sel(models=mod))
    
    tosssp245_ds_MAM_copy[i, :, :, :] = np.array(tosssp245_ds_MAM.sel(models=mod))
    tosssp245_ds_JJA_copy[i, :, :, :] = np.array(tosssp245_ds_JJA.sel(models=mod))
    tosssp245_ds_SON_copy[i, :, :, :] = np.array(tosssp245_ds_SON.sel(models=mod))
    tosssp245_ds_DJF_copy[i, :, :, :] = np.array(tosssp245_ds_DJF.sel(models=mod))
    
    hflsssp245_ds_JJA_copy[i, :, :, :] = np.array(hflsssp245_ds_JJA.sel(models=mod))

hgtssp245_ds_ver_JJA = hgtssp245_ds_ver_JJA_copy.copy()
ussp245_ds_ver_JJA = ussp245_ds_ver_JJA_copy.copy()
vssp245_ds_ver_JJA = vssp245_ds_ver_JJA_copy.copy()
qssp245_ds_ver_JJA = qssp245_ds_ver_JJA_copy.copy()
wapssp245_ds_ver_MAM = wapssp245_ds_ver_MAM_copy.copy()
wapssp245_ds_ver_JJA = wapssp245_ds_ver_JJA_copy.copy()
wapssp245_ds_ver_SON = wapssp245_ds_ver_SON_copy.copy()
wapssp245_ds_ver_DJF = wapssp245_ds_ver_DJF_copy.copy()
tassp245_ds_ver_JJA = tassp245_ds_ver_JJA_copy.copy()
spssp245_ds_JJA = spssp245_ds_JJA_copy.copy()
pressp245_ds_JJA = pressp245_ds_JJA_copy.copy()
tsssp245_ds_JJA = tsssp245_ds_JJA_copy.copy()
tosssp245_ds_MAM = tosssp245_ds_MAM_copy.copy()
tosssp245_ds_JJA = tosssp245_ds_JJA_copy.copy()
tosssp245_ds_SON = tosssp245_ds_SON_copy.copy()
tosssp245_ds_DJF = tosssp245_ds_DJF_copy.copy()
hflsssp245_ds_JJA = hflsssp245_ds_JJA_copy.copy()

hgtssp245_ds_ver_JJA.coords["models"] = models
ussp245_ds_ver_JJA.coords["models"] = models
vssp245_ds_ver_JJA.coords["models"] = models
qssp245_ds_ver_JJA.coords["models"] = models
wapssp245_ds_ver_MAM.coords["models"] = models
wapssp245_ds_ver_JJA.coords["models"] = models
wapssp245_ds_ver_SON.coords["models"] = models
wapssp245_ds_ver_DJF.coords["models"] = models
tassp245_ds_ver_JJA.coords["models"] = models
spssp245_ds_JJA.coords["models"] = models
pressp245_ds_JJA.coords["models"] = models
tsssp245_ds_JJA.coords["models"] = models
tosssp245_ds_MAM.coords["models"] = models
tosssp245_ds_JJA.coords["models"] = models
tosssp245_ds_SON.coords["models"] = models
tosssp245_ds_DJF.coords["models"] = models
hflsssp245_ds_JJA.coords["models"] = models


In [ ]:

hgtssp245_ds_ver_JJA.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/zg_ssp245_r144x72_201501-209912.nc")
ussp245_ds_ver_JJA.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/ua_ssp245_r144x72_201501-209912.nc")
vssp245_ds_ver_JJA.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/va_ssp245_r144x72_201501-209912.nc")
qssp245_ds_ver_JJA.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/hus_ssp245_r144x72_201501-209912.nc")
wapssp245_ds_ver_MAM.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/MAM/non_detrend/wap_ssp245_r144x72_201501-209912.nc")
wapssp245_ds_ver_JJA.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/wap_ssp245_r144x72_201501-209912.nc")
wapssp245_ds_ver_SON.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/SON/non_detrend/wap_ssp245_r144x72_201501-209912.nc")
wapssp245_ds_ver_DJF.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/DJF/non_detrend/wap_ssp245_r144x72_201501-209912.nc")
tassp245_ds_ver_JJA.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/ta_ssp245_r144x72_201501-209912.nc")
spssp245_ds_JJA.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/sp_ssp245_r144x72_201501-209912.nc")
pressp245_ds_JJA.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/pr_ssp245_r144x72_201501-209912.nc")
tsssp245_ds_JJA.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/ts_ssp245_r144x72_201501-209912.nc")
tosssp245_ds_MAM.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/MAM/non_detrend/tos_ssp245_r144x72_201501-209912.nc")
tosssp245_ds_JJA.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/tos_ssp245_r144x72_201501-209912.nc")
tosssp245_ds_SON.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/SON/non_detrend/tos_ssp245_r144x72_201501-209912.nc")
tosssp245_ds_DJF.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/DJF/non_detrend/tos_ssp245_r144x72_201501-209912.nc")
hflsssp245_ds_JJA.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/hfls_ssp245_r144x72_201501-209912.nc")


## 重新读取ssp245数据

In [ ]:
hgtssp245_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/zg_ssp245_r144x72_201501-209912.nc")
ussp245_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/ua_ssp245_r144x72_201501-209912.nc")
vssp245_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/va_ssp245_r144x72_201501-209912.nc")
qssp245_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/hus_ssp245_r144x72_201501-209912.nc")

wapssp245_ds_ver_MAM = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/MAM/non_detrend/wap_ssp245_r144x72_201501-209912.nc")
wapssp245_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/wap_ssp245_r144x72_201501-209912.nc")
wapssp245_ds_ver_SON = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/SON/non_detrend/wap_ssp245_r144x72_201501-209912.nc")
wapssp245_ds_ver_DJF = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/DJF/non_detrend/wap_ssp245_r144x72_201501-209912.nc")

tassp245_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/ta_ssp245_r144x72_201501-209912.nc")
pressp245_ds_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/pr_ssp245_r144x72_201501-209912.nc")
spssp245_ds_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/ps_ssp245_r144x72_201501-209912.nc")
tsssp245_ds_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/ts_ssp245_r144x72_201501-209912.nc")

tosssp245_ds_MAM = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/MAM/non_detrend/tos_ssp245_r144x72_201501-209912.nc")
tosssp245_ds_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/tos_ssp245_r144x72_201501-209912.nc")
tosssp245_ds_SON = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/SON/non_detrend/tos_ssp245_r144x72_201501-209912.nc")
tosssp245_ds_DJF = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/DJF/non_detrend/tos_ssp245_r144x72_201501-209912.nc")

hflsssp245_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/hfls_ssp245_r144x72_201501-209912.nc")

## 计算线性趋势

In [ ]:
hgtssp245_ds_ver_JJA_trend = hgtssp245_ds_ver_JJA.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)
ussp245_ds_ver_JJA_trend = ussp245_ds_ver_JJA.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)
vssp245_ds_ver_JJA_trend = vssp245_ds_ver_JJA.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)
qssp245_ds_ver_JJA_trend = qssp245_ds_ver_JJA.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)

wapssp245_ds_ver_MAM_trend = wapssp245_ds_ver_MAM.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)
wapssp245_ds_ver_JJA_trend = wapssp245_ds_ver_JJA.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)
wapssp245_ds_ver_SON_trend = wapssp245_ds_ver_SON.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)
wapssp245_ds_ver_DJF_trend = wapssp245_ds_ver_DJF.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)

tassp245_ds_ver_JJA_trend = tassp245_ds_ver_JJA.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)
pressp245_ds_JJA_trend = pressp245_ds_JJA.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)
spssp245_ds_JJA_trend = spssp245_ds_JJA.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)
tsssp245_ds_JJA_trend = tsssp245_ds_JJA.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)

tosssp245_ds_MAM_trend = tosssp245_ds_MAM.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)
tosssp245_ds_JJA_trend = tosssp245_ds_JJA.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)
tosssp245_ds_SON_trend = tosssp245_ds_SON.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)
tosssp245_ds_DJF_trend = tosssp245_ds_DJF.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)

hflsssp245_ds_ver_JJA_trend = hflsssp245_ds_ver_JJA.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)

In [ ]:
hgtssp245_ds_ver_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/hgtssp245_trend.nc")
ussp245_ds_ver_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/ussp245_trend.nc")
vssp245_ds_ver_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/vssp245_trend.nc")
qssp245_ds_ver_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/qssp245_trend.nc")

wapssp245_ds_ver_MAM_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/MAM/non_detrend/wapssp245_trend.nc")
wapssp245_ds_ver_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/wapssp245_trend.nc")
wapssp245_ds_ver_SON_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/SON/non_detrend/wapssp245_trend.nc")
wapssp245_ds_ver_DJF_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/DJF/non_detrend/wapssp245_trend.nc")

tassp245_ds_ver_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/tassp245_trend.nc")
pressp245_ds_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/pressp245_trend.nc")
spssp245_ds_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/spssp245_trend.nc")
tsssp245_ds_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/tsssp245_trend.nc")

tosssp245_ds_MAM_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/MAM/non_detrend/tosssp245_trend.nc")
tosssp245_ds_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/tosssp245_trend.nc")
tosssp245_ds_SON_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/SON/non_detrend/tosssp245_trend.nc")
tosssp245_ds_DJF_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/DJF/non_detrend/tosssp245_trend.nc")

hflsssp245_ds_ver_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/non_detrend/hflsssp245_trend.nc")

## 计算去趋势结果

In [ ]:
hgtssp245_ds_ver_JJA_detrend = ca.detrend_dim(hgtssp245_ds_ver_JJA, "time", deg=1, demean=True)
ussp245_ds_ver_JJA_detrend = ca.detrend_dim(ussp245_ds_ver_JJA, "time", deg=1, demean=True)
vssp245_ds_ver_JJA_detrend = ca.detrend_dim(vssp245_ds_ver_JJA, "time", deg=1, demean=True)
qssp245_ds_ver_JJA_detrend = ca.detrend_dim(qssp245_ds_ver_JJA, "time", deg=1, demean=True)

wapssp245_ds_ver_MAM_detrend = ca.detrend_dim(wapssp245_ds_ver_MAM, "time", deg=1, demean=True)
wapssp245_ds_ver_JJA_detrend = ca.detrend_dim(wapssp245_ds_ver_JJA, "time", deg=1, demean=True)
wapssp245_ds_ver_SON_detrend = ca.detrend_dim(wapssp245_ds_ver_SON, "time", deg=1, demean=True)
wapssp245_ds_ver_DJF_detrend = ca.detrend_dim(wapssp245_ds_ver_DJF, "time", deg=1, demean=True)

tassp245_ds_ver_JJA_detrend = ca.detrend_dim(tassp245_ds_ver_JJA, "time", deg=1, demean=True)
spssp245_ds_JJA_detrend = ca.detrend_dim(spssp245_ds_JJA, "time", deg=1, demean=True)
pressp245_ds_JJA_detrend = ca.detrend_dim(pressp245_ds_JJA, "time", deg=1, demean=True)
tsssp245_ds_JJA_detrend = ca.detrend_dim(tsssp245_ds_JJA, "time", deg=1, demean=True)

tosssp245_ds_MAM_detrend = ca.detrend_dim(tosssp245_ds_MAM, "time", deg=1, demean=True)
tosssp245_ds_JJA_detrend = ca.detrend_dim(tosssp245_ds_JJA, "time", deg=1, demean=True)
tosssp245_ds_SON_detrend = ca.detrend_dim(tosssp245_ds_SON, "time", deg=1, demean=True)
tosssp245_ds_DJF_detrend = ca.detrend_dim(tosssp245_ds_DJF, "time", deg=1, demean=True)

hflsssp245_ds_ver_JJA_detrend = ca.detrend_dim(hflsssp245_ds_ver_JJA, "time", deg=1, demean=True)

In [ ]:
hgtssp245_ds_ver_JJA_detrend.name = "zg"
ussp245_ds_ver_JJA_detrend.name = "ua"
vssp245_ds_ver_JJA_detrend.name = "va"
qssp245_ds_ver_JJA_detrend.name = "hus"

wapssp245_ds_ver_MAM_detrend.name = "wap"
wapssp245_ds_ver_JJA_detrend.name = "wap"
wapssp245_ds_ver_SON_detrend.name = "wap"
wapssp245_ds_ver_DJF_detrend.name = "wap"

tassp245_ds_ver_JJA_detrend.name = "ta"
spssp245_ds_JJA_detrend.name = "ps"
pressp245_ds_JJA_detrend.name = "pr"
tsssp245_ds_JJA_detrend.name = "ts"

tosssp245_ds_MAM_detrend.name = "sst"
tosssp245_ds_JJA_detrend.name = "sst"
tosssp245_ds_SON_detrend.name = "sst"
tosssp245_ds_DJF_detrend.name = "sst"

hflsssp245_ds_ver_JJA_detrend.name = "hfls"

In [ ]:
hgtssp245_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/zg_ssp245_r144x72_201501-209912.nc")
ussp245_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/ua_ssp245_r144x72_201501-209912.nc")
vssp245_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/va_ssp245_r144x72_201501-209912.nc")
qssp245_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/hus_ssp245_r144x72_201501-209912.nc")
wapssp245_ds_ver_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/MAM/detrend/wap_ssp245_r144x72_201501-209912.nc")
wapssp245_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/wap_ssp245_r144x72_201501-209912.nc")
wapssp245_ds_ver_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/SON/detrend/wap_ssp245_r144x72_201501-209912.nc")
wapssp245_ds_ver_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/DJF/detrend/wap_ssp245_r144x72_201501-209912.nc")
tassp245_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/ta_ssp245_r144x72_201501-209912.nc")
spssp245_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/ps_ssp245_r144x72_201501-209912.nc")
pressp245_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/pr_ssp245_r144x72_201501-209912.nc")
tsssp245_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/ts_ssp245_r144x72_201501-209912.nc")
tosssp245_ds_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/MAM/detrend/tos_ssp245_r144x72_201501-209912.nc")
tosssp245_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/tos_ssp245_r144x72_201501-209912.nc")
tosssp245_ds_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/SON/detrend/tos_ssp245_r144x72_201501-209912.nc")
tosssp245_ds_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/DJF/detrend/tos_ssp245_r144x72_201501-209912.nc")
hflsssp245_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/hfls_ssp245_r144x72_201501-209912.nc")

In [ ]:
hgtssp245_p3_ds_ver_JJA_detrend = ca.detrend_dim(hgtssp245_ds_ver_JJA_detrend.sel(time=hgtssp245_ds_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
ussp245_p3_ds_ver_JJA_detrend = ca.detrend_dim(ussp245_ds_ver_JJA_detrend.sel(time=ussp245_ds_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
vssp245_p3_ds_ver_JJA_detrend = ca.detrend_dim(vssp245_ds_ver_JJA_detrend.sel(time=vssp245_ds_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
qssp245_p3_ds_ver_JJA_detrend = ca.detrend_dim(qssp245_ds_ver_JJA_detrend.sel(time=qssp245_ds_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

wapssp245_p3_ds_ver_MAM_detrend = ca.detrend_dim(wapssp245_ds_ver_MAM_detrend.sel(time=wapssp245_ds_ver_MAM_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

wapssp245_p3_ds_ver_JJA_detrend = ca.detrend_dim(wapssp245_ds_ver_JJA_detrend.sel(time=wapssp245_ds_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

wapssp245_p3_ds_ver_SON_detrend = ca.detrend_dim(wapssp245_ds_ver_SON_detrend.sel(time=wapssp245_ds_ver_SON_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

wapssp245_p3_ds_ver_DJF_detrend = ca.detrend_dim(wapssp245_ds_ver_DJF_detrend.sel(time=wapssp245_ds_ver_DJF_detrend.time.dt.year>=2064), "time", deg=1, demean=True)


tassp245_p3_ds_ver_JJA_detrend = ca.detrend_dim(tassp245_ds_ver_JJA_detrend.sel(time=tassp245_ds_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
spssp245_p3_ds_JJA_detrend = ca.detrend_dim(spssp245_ds_JJA_detrend.sel(time=spssp245_ds_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
pressp245_p3_ds_JJA_detrend = ca.detrend_dim(pressp245_ds_JJA_detrend.sel(time=pressp245_ds_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
tsssp245_p3_ds_JJA_detrend = ca.detrend_dim(tsssp245_ds_JJA_detrend.sel(time=tsssp245_ds_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

tosssp245_p3_ds_MAM_detrend = ca.detrend_dim(tosssp245_ds_MAM_detrend.sel(time=tosssp245_ds_MAM_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

tosssp245_p3_ds_JJA_detrend = ca.detrend_dim(tosssp245_ds_JJA_detrend.sel(time=tosssp245_ds_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

tosssp245_p3_ds_SON_detrend = ca.detrend_dim(tosssp245_ds_SON_detrend.sel(time=tosssp245_ds_SON_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

tosssp245_p3_ds_DJF_detrend = ca.detrend_dim(tosssp245_ds_DJF_detrend.sel(time=tosssp245_ds_DJF_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

hflsssp245_p3_ds_ver_JJA_detrend = ca.detrend_dim(hflsssp245_ds_ver_JJA_detrend.sel(time=hflsssp245_ds_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

In [ ]:
hgtssp245_p3_ds_ver_JJA_detrend.name = "zg"
ussp245_p3_ds_ver_JJA_detrend.name = "ua"
vssp245_p3_ds_ver_JJA_detrend.name = "va"
qssp245_p3_ds_ver_JJA_detrend.name = "hus"

wapssp245_p3_ds_ver_MAM_detrend.name = "wap"
wapssp245_p3_ds_ver_JJA_detrend.name = "wap"
wapssp245_p3_ds_ver_SON_detrend.name = "wap"
wapssp245_p3_ds_ver_DJF_detrend.name = "wap"

tassp245_p3_ds_ver_JJA_detrend.name = "ta"
spssp245_p3_ds_JJA_detrend.name = "ps"
pressp245_p3_ds_JJA_detrend.name = "pr"
tsssp245_p3_ds_JJA_detrend.name = "ts"

tosssp245_p3_ds_MAM_detrend.name = "sst"
tosssp245_p3_ds_JJA_detrend.name = "sst"
tosssp245_p3_ds_SON_detrend.name = "sst"
tosssp245_p3_ds_DJF_detrend.name = "sst"

hflsssp245_p3_ds_ver_JJA_detrend.name = "hfls"

In [ ]:
hgtssp245_p3_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/zg_ssp245_p3_r144x72_206401-209912.nc")
ussp245_p3_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/ua_ssp245_p3_r144x72_206401-209912.nc")
vssp245_p3_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/va_ssp245_p3_r144x72_206401-209912.nc")
qssp245_p3_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/hus_ssp245_p3_r144x72_206401-209912.nc")

wapssp245_p3_ds_ver_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/MAM/detrend/wap_ssp245_p3_r144x72_206401-209912.nc")
wapssp245_p3_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/wap_ssp245_p3_r144x72_206401-209912.nc")
wapssp245_p3_ds_ver_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/SON/detrend/wap_ssp245_p3_r144x72_206401-209912.nc")
wapssp245_p3_ds_ver_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/DJF/detrend/wap_ssp245_p3_r144x72_206401-209912.nc")

tassp245_p3_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/ta_ssp245_p3_r144x72_206401-209912.nc")
pressp245_p3_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/pr_ssp245_p3_r144x72_206401-209912.nc")
spssp245_p3_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/ps_ssp245_p3_r144x72_206401-209912.nc")
tsssp245_p3_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/ts_ssp245_p3_r144x72_206401-209912.nc")

tosssp245_p3_ds_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/MAM/detrend/tos_ssp245_p3_r144x72_206401-209912.nc")
tosssp245_p3_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/tos_ssp245_p3_r144x72_206401-209912.nc")
tosssp245_p3_ds_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/SON/detrend/tos_ssp245_p3_r144x72_206401-209912.nc")
tosssp245_p3_ds_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/DJF/detrend/tos_ssp245_p3_r144x72_206401-209912.nc")

hflsssp245_p3_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/tmp_var/JJA/detrend/hfls_ssp245_p3_r144x72_206401-209912.nc")

# 对ssp585数据去除长期趋势

## 读取ssp585的数据

In [ ]:
hgtssp585_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/zg_ssp585_r144x72_201501-209912.nc")
ussp585_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/ua_ssp585_r144x72_201501-209912.nc")
vssp585_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/va_ssp585_r144x72_201501-209912.nc")
qssp585_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/hus_ssp585_r144x72_201501-209912.nc")
wapssp585_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/wap_ssp585_r144x72_201501-209912.nc")
tassp585_ds_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/ta_ssp585_r144x72_201501-209912.nc")
pressp585_ds_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/pr_ssp585_r144x72_201501-209912.nc")
spssp585_ds_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/ps_ssp585_r144x72_201501-209912.nc")
tsssp585_ds_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/ts_ssp585_r144x72_201501-209912.nc")
tosssp585_ds_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/tos_ssp585_r144x72_201501-209912.nc")
hflsssp585_ds_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/hfls_ssp585_r144x72_201501-209912.nc")

In [ ]:
tosssp585_ds_MAM = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/non_detrend/tos_ssp585_r144x72_201501-209912.nc")
tosssp585_ds_SON = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/non_detrend/tos_ssp585_r144x72_201501-209912.nc")
tosssp585_ds_DJF = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/non_detrend/tos_ssp585_r144x72_201501-209912.nc")

wapssp585_ds_MAM = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/non_detrend/wap_ssp585_r144x72_201501-209912.nc")
wapssp585_ds_SON = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/non_detrend/wap_ssp585_r144x72_201501-209912.nc")
wapssp585_ds_DJF = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/non_detrend/wap_ssp585_r144x72_201501-209912.nc")

## 重新计算去趋势结果

In [ ]:
hgtssp585_ds_ver_JJA_detrend = ca.detrend_dim(hgtssp585_ds_ver_JJA)
ussp585_ds_ver_JJA_detrend = ca.detrend_dim(ussp585_ds_ver_JJA)
vssp585_ds_ver_JJA_detrend = ca.detrend_dim(vssp585_ds_ver_JJA)
qssp585_ds_ver_JJA_detrend = ca.detrend_dim(qssp585_ds_ver_JJA)
wapssp585_ds_ver_JJA_detrend = ca.detrend_dim(wapssp585_ds_ver_JJA)
tassp585_ds_ver_JJA_detrend = ca.detrend_dim(tassp585_ds_ver_JJA)
pressp585_ds_JJA_detrend = ca.detrend_dim(pressp585_ds_JJA)
spssp585_ds_JJA_detrend = ca.detrend_dim(spssp585_ds_JJA)
tsssp585_ds_JJA_detrend = ca.detrend_dim(tsssp585_ds_JJA)
tosssp585_ds_JJA_detrend = ca.detrend_dim(tosssp585_ds_JJA)
hflsssp585_ds_JJA_detrend = ca.detrend_dim(hflsssp585_ds_JJA)

In [ ]:
tosssp585_ds_MAM_detrend = ca.detrend_dim(tosssp585_ds_MAM)
tosssp585_ds_SON_detrend = ca.detrend_dim(tosssp585_ds_SON)
tosssp585_ds_DJF_detrend = ca.detrend_dim(tosssp585_ds_DJF)

wapssp585_ds_MAM_detrend = ca.detrend_dim(wapssp585_ds_MAM)
wapssp585_ds_SON_detrend = ca.detrend_dim(wapssp585_ds_SON)
wapssp585_ds_DJF_detrend = ca.detrend_dim(wapssp585_ds_DJF)

In [ ]:
hgtssp585_ds_ver_JJA_detrend.name = "zg"
ussp585_ds_ver_JJA_detrend.name = "ua"
vssp585_ds_ver_JJA_detrend.name = "va"
qssp585_ds_ver_JJA_detrend.name = "hus"
wapssp585_ds_ver_JJA_detrend.name = "wap"
tassp585_ds_ver_JJA_detrend.name = "ta"
pressp585_ds_JJA_detrend.name = "pr"
spssp585_ds_JJA_detrend.name = "sp"
tsssp585_ds_JJA_detrend.name = "ts"
tosssp585_ds_JJA_detrend.name = "tos"
hflsssp585_ds_JJA_detrend.name = "hfls"

In [ ]:
tosssp585_ds_MAM_detrend.name = "tos"
tosssp585_ds_SON_detrend.name = "tos"
tosssp585_ds_DJF_detrend.name = "tos"
wapssp585_ds_MAM_detrend.name = "wap"
wapssp585_ds_SON_detrend.name = "wap"
wapssp585_ds_DJF_detrend.name = "wap"

In [ ]:
hgtssp585_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/hgt_ssp585_r144x72_detrend.nc")
ussp585_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/u_ssp585_r144x72_detrend.nc")
vssp585_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/v_ssp585_r144x72_detrend.nc")
qssp585_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/q_ssp585_r144x72_detrend.nc")
wapssp585_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/wap_ssp585_r144x72_detrend.nc")
tassp585_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ta_ssp585_r144x72_detrend.nc")
pressp585_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/pre_ssp585_r144x72_detrend.nc")
spssp585_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/sp_ssp585_r144x72_detrend.nc")
tsssp585_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ts_ssp585_r144x72_detrend.nc")
tosssp585_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/tos_ssp585_r144x72_detrend.nc")
hflsssp585_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/hfls_ssp585_r144x72_detrend.nc")

In [ ]:
tosssp585_ds_MAM.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/tos_ssp585_r144x72_detrend.nc")
tosssp585_ds_SON.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/tos_ssp585_r144x72_detrend.nc")
tosssp585_ds_DJF.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/tos_ssp585_r144x72_detrend.nc")

wapssp585_ds_MAM.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/wap_ssp585_r144x72_detrend.nc")
wapssp585_ds_SON.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/wap_ssp585_r144x72_detrend.nc")
wapssp585_ds_DJF.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/wap_ssp585_r144x72_detrend.nc")

In [ ]:
hgtssp585_p3_ds_ver_JJA_detrend = ca.detrend_dim(hgtssp585_ds_ver_JJA_detrend.sel(time=hgtssp585_ds_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
ussp585_p3_ds_ver_JJA_detrend = ca.detrend_dim(ussp585_ds_ver_JJA_detrend.sel(time=ussp585_ds_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
vssp585_p3_ds_ver_JJA_detrend = ca.detrend_dim(vssp585_ds_ver_JJA_detrend.sel(time=vssp585_ds_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
qssp585_p3_ds_ver_JJA_detrend = ca.detrend_dim(qssp585_ds_ver_JJA_detrend.sel(time=qssp585_ds_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

wapssp585_p3_ds_MAM_detrend = ca.detrend_dim(wapssp585_ds_MAM_detrend.sel(time=wapssp585_ds_MAM_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

wapssp585_p3_ds_ver_JJA_detrend = ca.detrend_dim(wapssp585_ds_ver_JJA_detrend.sel(time=wapssp585_ds_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

wapssp585_p3_ds_SON_detrend = ca.detrend_dim(wapssp585_ds_SON_detrend.sel(time=wapssp585_ds_SON_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

wapssp585_p3_ds_DJF_detrend = ca.detrend_dim(wapssp585_ds_DJF_detrend.sel(time=wapssp585_ds_DJF_detrend.time.dt.year>=2064), "time", deg=1, demean=True)


tassp585_p3_ds_ver_JJA_detrend = ca.detrend_dim(tassp585_ds_ver_JJA_detrend.sel(time=tassp585_ds_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
spssp585_p3_ds_JJA_detrend = ca.detrend_dim(spssp585_ds_JJA_detrend.sel(time=spssp585_ds_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
pressp585_p3_ds_JJA_detrend = ca.detrend_dim(pressp585_ds_JJA_detrend.sel(time=pressp585_ds_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
tsssp585_p3_ds_JJA_detrend = ca.detrend_dim(tsssp585_ds_JJA_detrend.sel(time=tsssp585_ds_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

tosssp585_p3_ds_MAM_detrend = ca.detrend_dim(tosssp585_ds_MAM_detrend.sel(time=tosssp585_ds_MAM_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

tosssp585_p3_ds_JJA_detrend = ca.detrend_dim(tosssp585_ds_JJA_detrend.sel(time=tosssp585_ds_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

tosssp585_p3_ds_SON_detrend = ca.detrend_dim(tosssp585_ds_SON_detrend.sel(time=tosssp585_ds_SON_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

tosssp585_p3_ds_DJF_detrend = ca.detrend_dim(tosssp585_ds_DJF_detrend.sel(time=tosssp585_ds_DJF_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

hflsssp585_p3_ds_JJA_detrend = ca.detrend_dim(hflsssp585_ds_JJA_detrend.sel(time=hflsssp585_ds_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

In [ ]:
hgtssp585_p3_ds_ver_JJA_detrend.name = "zg"
ussp585_p3_ds_ver_JJA_detrend.name = "ua"
vssp585_p3_ds_ver_JJA_detrend.name = "va"
qssp585_p3_ds_ver_JJA_detrend.name = "hus"

wapssp585_p3_ds_MAM_detrend.name = "wap"
wapssp585_p3_ds_ver_JJA_detrend.name = "wap"
wapssp585_p3_ds_SON_detrend.name = "wap"
wapssp585_p3_ds_DJF_detrend.name = "wap"

tassp585_p3_ds_ver_JJA_detrend.name = "ta"
spssp585_p3_ds_JJA_detrend.name = "ps"
pressp585_p3_ds_JJA_detrend.name = "pr"
tsssp585_p3_ds_JJA_detrend.name = "ts"

tosssp585_p3_ds_MAM_detrend.name = "sst"
tosssp585_p3_ds_JJA_detrend.name = "sst"
tosssp585_p3_ds_SON_detrend.name = "sst"
tosssp585_p3_ds_DJF_detrend.name = "sst"

hflsssp585_p3_ds_JJA_detrend.name = "hfls"

In [ ]:
hgtssp585_p3_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/zg_ssp585_p3_r144x72_detrend.nc")
ussp585_p3_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ua_ssp585_p3_r144x72_detrend.nc")
vssp585_p3_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/va_ssp585_p3_r144x72_detrend.nc")
qssp585_p3_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/hus_ssp585_p3_r144x72_detrend.nc")

wapssp585_p3_ds_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/wap_ssp585_p3_r144x72_detrend.nc")
wapssp585_p3_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/wap_ssp585_p3_r144x72_detrend.nc")
wapssp585_p3_ds_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/wap_ssp585_p3_r144x72_detrend.nc")
wapssp585_p3_ds_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/wap_ssp585_p3_r144x72_detrend.nc")

tassp585_p3_ds_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ta_ssp585_p3_r144x72_detrend.nc")
pressp585_p3_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/pr_ssp585_p3_r144x72_detrend.nc")
spssp585_p3_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ps_ssp585_p3_r144x72_detrend.nc")
tsssp585_p3_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ts_ssp585_p3_r144x72_detrend.nc")

tosssp585_p3_ds_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/tos_ssp585_p3_r144x72_detrend.nc")
tosssp585_p3_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/tos_ssp585_p3_r144x72_detrend.nc")
tosssp585_p3_ds_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/tos_ssp585_p3_r144x72_detrend.nc")
tosssp585_p3_ds_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/tos_ssp585_p3_r144x72_detrend.nc")

hflsssp585_p3_ds_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/hfls_ssp585_p3_r144x72_detrend.nc")